In [6]:
#Подключение к БД на SQLite 

import pandas as pd
import numpy as np

from sqlalchemy import create_engine

path_to_db = 'telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [7]:
query = """
WITH
    u AS
        (SELECT user_id,
                location_id,
                age_gr_id,
                tr_gr_id,
                lt_gr_id,
            lt_day,
                (CASE
                    WHEN lt_day < 365 THEN "'новые' клиенты" 
                    WHEN lt_day >= 365 THEN "'старые' клиенты"
                END) AS is_new,
                age,
                (CASE
                    WHEN gender_segment = 1 THEN 'женщины' 
                    WHEN gender_segment = 0 THEN 'мужчины'
                    ELSE 'не указан'
                END) AS gender_segment,
                os_name,
                cpe_type_name,
                nps_score,
                (CASE
                    WHEN nps_score IN (9,10) THEN 'promoters' 
                    WHEN nps_score IN (7,8) THEN 'passives'
                    WHEN nps_score IN (0,1,2,3,4,5,6) THEN 'detractors'
                    ELSE 'неизвестно'
                END) AS nps_group
            FROM user),
    a AS
        (SELECT age_gr_id,
                substr(title,3) AS age_segment
        FROM age_segment),
    t AS
        (SELECT tr_gr_id,
                substr(title,3) AS traffic_segment
        FROM traffic_segment),
    ls AS
        (SELECT lt_gr_id,
                substr(title,3) AS lifetime_segment
        FROM lifetime_segment)
        
SELECT
    u.user_id,
    u.lt_day,
    u.is_new,
    u.age,
    u.gender_segment,
    u.os_name,
    u.cpe_type_name,
    u.nps_score,
    u.nps_group,
    location.city,
    location.country,
    a.age_segment,
    t.traffic_segment,
    ls.lifetime_segment
FROM u 
    LEFT OUTER JOIN location ON u.location_id=location.location_id
    LEFT OUTER JOIN a ON u.age_gr_id=a.age_gr_id
    LEFT OUTER JOIN t ON u.tr_gr_id=t.tr_gr_id
    LEFT OUTER JOIN ls ON u.lt_gr_id=ls.lt_gr_id

"""

In [8]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,nps_score,nps_group,city,country,age_segment,traffic_segment,lifetime_segment
0,A001A2,2320,'старые' клиенты,45.0,женщины,ANDROID,SMARTPHONE,10,promoters,Уфа,Россия,45-54,1-5,36+
1,A001WF,2344,'старые' клиенты,53.0,мужчины,ANDROID,SMARTPHONE,10,promoters,Киров,Россия,45-54,1-5,36+
2,A003Q7,467,'старые' клиенты,57.0,мужчины,ANDROID,SMARTPHONE,10,promoters,Москва,Россия,55-64,20-25,13-24
3,A004TB,4190,'старые' клиенты,44.0,женщины,IOS,SMARTPHONE,10,promoters,РостовнаДону,Россия,35-44,0.1-1,36+
4,A004XT,1163,'старые' клиенты,24.0,мужчины,ANDROID,SMARTPHONE,10,promoters,Рязань,Россия,16-24,5-10,36+
5,A005O0,5501,'старые' клиенты,42.0,женщины,ANDROID,SMARTPHONE,6,detractors,Омск,Россия,35-44,5-10,36+
6,A0061R,1236,'старые' клиенты,45.0,мужчины,ANDROID,SMARTPHONE,10,promoters,Уфа,Россия,45-54,10-15,36+
7,A009KS,313,'новые' клиенты,35.0,мужчины,ANDROID,SMARTPHONE,10,promoters,Москва,Россия,35-44,45-50,7-12
8,A00AES,3238,'старые' клиенты,36.0,женщины,ANDROID,SMARTPHONE,10,promoters,СанктПетербург,Россия,35-44,1-5,36+
9,A00F70,4479,'старые' клиенты,54.0,женщины,ANDROID,SMARTPHONE,9,promoters,Волгоград,Россия,45-54,15-20,36+


In [9]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Ссылка на дашборд Tableau Public:

https://public.tableau.com/views/project_2_16509148990350/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link